# I - Urban/Rural Designation and County Elections

In [1]:
import pandas as pd
import numpy as np
import altair as alt

## A- County Urban/Rural Designation

In [2]:
# Note: Data input file path

CountyUrbanRural = pd.read_excel('../DataForPresidentialElectionsAndCovid/County_Rural_Lookup.xlsx',
                                 skiprows=3, usecols='A:H')

CountyUrbanRural.head()

,2015 GEOID,State,2015 Geography Name,Note,2010 Census Total Population,2010 Census Urban Population,2010 Census Rural Population,2010 Census \nPercent Rural
0,01001,AL,"Autauga County, Alabama",NaN,54571.0,31650.0,22921.0,42.002162
1,01003,AL,"Baldwin County, Alabama",NaN,182265.0,105205.0,77060.0,42.279099
2,01005,AL,"Barbour County, Alabama",NaN,27457.0,8844.0,18613.0,67.789635
3,01007,AL,"Bibb County, Alabama",NaN,22915.0,7252.0,15663.0,68.352607
4,01009,AL,"Blount County, Alabama",NaN,57322.0,5760.0,51562.0,89.951502


In [3]:
# Drop last six rows of footnotes
CountyUrbanRural = CountyUrbanRural[:-6]

# Confirm last row is data
CountyUrbanRural['2015 GEOID'].iloc[-1]

'56045'

In [4]:
# According to the input file notes, a county is considered urban if it is
# less than 50% rural. The Rural designation is mainly based on population density.

# Add urban/rural column and retain percentage for later use in charts.
CountyUrbanRural['UrbanRural'] = CountyUrbanRural['2010 Census \nPercent Rural'] \
                                 .map(lambda x: 'urban' if x<50 else 'rural')

CountyUrbanRural = CountyUrbanRural[['2015 GEOID', 'UrbanRural', '2010 Census \nPercent Rural']]
CountyUrbanRural.head()

,2015 GEOID,UrbanRural,2010 Census \nPercent Rural
0,01001,urban,42.002162
1,01003,urban,42.279099
2,01005,rural,67.789635
3,01007,rural,68.352607
4,01009,rural,89.951502


In [5]:

CountyUrbanRural.rename(columns={'2015 GEOID': 'county_fips',
                                 '2010 Census \nPercent Rural': 'PctRural'}, inplace=True)
CountyUrbanRural.head()

,county_fips,UrbanRural,PctRural
0,01001,urban,42.002162
1,01003,urban,42.279099
2,01005,rural,67.789635
3,01007,rural,68.352607
4,01009,rural,89.951502


In [6]:
# Check data types
print(type(CountyUrbanRural['county_fips'].iloc[0]))
print(type(CountyUrbanRural['UrbanRural'].iloc[0]))
print(type(CountyUrbanRural['PctRural'].iloc[0]))

<class 'str'>
<class 'str'>
<class 'numpy.float64'>


In [7]:
CountyUrbanRural['county_fips'] = CountyUrbanRural['county_fips'].astype(int)

In [8]:
CountyUrbanRural.shape

(3142, 3)

## B- County Election Data

In [9]:
# Election data by county 2000-2020
# Note: Data input file path

PECountyDF = pd.read_csv('../DataForPresidentialElectionsAndCovid/countypres_2000-2020.csv')
PECountyDF.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,AL GORE,DEMOCRAT,4942.0,17208.0,20191203,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993.0,17208.0,20191203,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,RALPH NADER,GREEN,160.0,17208.0,20191203,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,OTHER,OTHER,113.0,17208.0,20191203,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003.0,PRESIDENT,AL GORE,DEMOCRAT,13997.0,56480.0,20191203,TOTAL


In [10]:
PECountyDF.shape

(72603, 12)

In [11]:
# Only interested in 2020

PECountyDF = PECountyDF[PECountyDF['year'] == 2020]
PECountyDF.shape

(22079, 12)

In [12]:
# Choose columns of interest.
# Don't really need the candidate, but keep for later sanity checks and counts.

PECountyDF = PECountyDF[['state_po', 'county_name', 'county_fips', 'candidate', 'party',
                         'candidatevotes','totalvotes']]
PECountyDF.head(10)

,state_po,county_name,county_fips,candidate,party,candidatevotes,totalvotes
50524,AL,AUTAUGA,1001.0,JOSEPH R BIDEN JR,DEMOCRAT,7503.0,27770.0
50525,AL,AUTAUGA,1001.0,NaN,OTHER,429.0,27770.0
50526,AL,AUTAUGA,1001.0,DONALD J TRUMP,REPUBLICAN,19838.0,27770.0
50527,AL,BALDWIN,1003.0,JOSEPH R BIDEN JR,DEMOCRAT,24578.0,109679.0
50528,AL,BALDWIN,1003.0,JO JORGENSEN,OTHER,1557.0,109679.0
50529,AL,BALDWIN,1003.0,DONALD J TRUMP,REPUBLICAN,83544.0,109679.0
50530,AL,BARBOUR,1005.0,JOSEPH R BIDEN JR,DEMOCRAT,4816.0,10518.0
50531,AL,BARBOUR,1005.0,JO JORGENSEN,OTHER,80.0,10518.0
50532,AL,BARBOUR,1005.0,DONALD J TRUMP,REPUBLICAN,5622.0,10518.0
50533,AL,BIBB,1007.0,JOSEPH R BIDEN JR,DEMOCRAT,1986.0,9595.0


In [13]:
# Only need the winning party
# Source: https://stackoverflow.com/questions/15705630/ \
#                 get-the-rows-which-have-the-max-value-in-groups-using-groupby

idx = PECountyDF.groupby(['state_po', 'county_name', 'county_fips', 'totalvotes'])['candidatevotes'] \
                .transform(max) == PECountyDF['candidatevotes']

PECountyDF = PECountyDF[idx]
PECountyDF.head()

,state_po,county_name,county_fips,candidate,party,candidatevotes,totalvotes
50526,AL,AUTAUGA,1001.0,DONALD J TRUMP,REPUBLICAN,19838.0,27770.0
50529,AL,BALDWIN,1003.0,DONALD J TRUMP,REPUBLICAN,83544.0,109679.0
50532,AL,BARBOUR,1005.0,DONALD J TRUMP,REPUBLICAN,5622.0,10518.0
50535,AL,BIBB,1007.0,DONALD J TRUMP,REPUBLICAN,7525.0,9595.0
50538,AL,BLOUNT,1009.0,DONALD J TRUMP,REPUBLICAN,24711.0,27588.0


In [14]:
PECountyDF.shape

(3152, 7)

### 1. Clean the county FIPS codes

In [15]:
# Need to check this for the merge later

type(PECountyDF['county_fips'].iloc[0])


numpy.float64

In [16]:
# Convert to int
PECountyDF['county_fips'] = PECountyDF['county_fips'].astype(int)
type(PECountyDF['county_fips'].iloc[0])

numpy.int64

### 2. Clean up several data issues

In [17]:
# See what effect an unknown candidate/party has. Won't be able
# to determine conservative/liberal leaning, so see if dropping
# them has a significant effect. There are no such rows.

TempDF0 = PECountyDF[PECountyDF['party'] == 'OTHER']
TempDF0

,state_po,county_name,county_fips,candidate,party,candidatevotes,totalvotes
66639,VT,ADDISON,50001,"ROQUE ""ROCKY"" DE LA FUENTE",OTHER,15727.0,22611.0
66643,VT,BENNINGTON,50003,PATRICK ODONNELL,OTHER,12371.0,19357.0
66647,VT,CALEDONIA,50005,DAVE MARX/ JOSH DOUBLEDAY,OTHER,9020.0,15158.0
66651,VT,CHITTENDEN,50007,JIM DOUGLAS,OTHER,75435.0,96755.0
66659,VT,FRANKLIN,50011,PETE BUTTEGIEG,OTHER,14404.0,26072.0
66663,VT,GRAND ISLE,50013,BRYAN LIACCIA,OTHER,3528.0,5782.0
66667,VT,LAMOILLE,50015,JOHN KEBR,OTHER,11281.0,15679.0
66671,VT,ORANGE,50017,SPOILED VOTES,OTHER,8890.0,14876.0
66675,VT,ORLEANS,50019,ZADA GILMORE,OTHER,8174.0,15442.0
66679,VT,RUTLAND,50021,GILBERT FITZHUGH,OTHER,19711.0,34772.0


In [18]:
# Drop these counties since political ideology can't be easily determined.

PECountyDF = PECountyDF[PECountyDF['party'] != 'OTHER']
PECountyDF.shape

(3139, 7)

In [19]:
# Some states allow candidates to appear on multiple party lines.
# See if that occurs in selected data, and where. None remaining
# after losing candidates were excluded.

TempDF1 = PECountyDF[PECountyDF['candidate'] != 'OTHER']

TempDF1 = TempDF1[['state_po', 'county_fips', 'candidate', 'party']] \
                        .groupby(['state_po', 'county_fips', 'candidate']).count()
TempDF1 = TempDF1[TempDF1['party'] != 1]
TempDF1.head()

,,,party
state_po,county_fips,candidate,


## C- Merge the dataframes

In [20]:
PEUrbanRuralDF = PECountyDF.merge(CountyUrbanRural, on='county_fips', how='inner')
PEUrbanRuralDF.head()

,state_po,county_name,county_fips,candidate,party,candidatevotes,totalvotes,UrbanRural,PctRural
0,AL,AUTAUGA,1001,DONALD J TRUMP,REPUBLICAN,19838.0,27770.0,urban,42.002162
1,AL,BALDWIN,1003,DONALD J TRUMP,REPUBLICAN,83544.0,109679.0,urban,42.279099
2,AL,BARBOUR,1005,DONALD J TRUMP,REPUBLICAN,5622.0,10518.0,rural,67.789635
3,AL,BIBB,1007,DONALD J TRUMP,REPUBLICAN,7525.0,9595.0,rural,68.352607
4,AL,BLOUNT,1009,DONALD J TRUMP,REPUBLICAN,24711.0,27588.0,rural,89.951502


In [21]:
PEUrbanRuralDF.shape

(3099, 9)

In [22]:
# See how many rows were lost in the merge
MissingDF = (PECountyDF.merge(PEUrbanRuralDF, on='county_fips', how='left', indicator=True)
                      .query('_merge == "left_only"')
                      .drop('_merge', 1))
MissingDF.shape

(40, 15)

In [23]:
MissingDF['county_fips'].unique()

array([ 2001,  2002,  2003,  2004,  2005,  2006,  2007,  2008,  2009,
        2010,  2011,  2012,  2014,  2015,  2017,  2018,  2019,  2021,
        2022,  2023,  2024,  2025,  2026,  2027,  2028,  2029,  2030,
        2031,  2032,  2033,  2034,  2035,  2036,  2037,  2038,  2039,
        2040,  2099, 36000, 46113])

In [24]:
MissingDF.head()

,state_po_x,county_name_x,county_fips,candidate_x,party_x,candidatevotes_x,totalvotes_x,state_po_y,county_name_y,candidate_y,party_y,candidatevotes_y,totalvotes_y,UrbanRural,PctRural
67,AK,DISTRICT 1,2001,DONALD J TRUMP,REPUBLICAN,3511.0,7314.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,AK,DISTRICT 2,2002,DONALD J TRUMP,REPUBLICAN,3674.0,6136.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,AK,DISTRICT 3,2003,DONALD J TRUMP,REPUBLICAN,6076.0,8385.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,AK,DISTRICT 4,2004,JOSEPH R BIDEN JR,DEMOCRAT,5345.0,10511.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,AK,DISTRICT 5,2005,JOSEPH R BIDEN JR,DEMOCRAT,4204.0,8654.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
#Drop the duplicate columns
MissingDF.drop([col for col in MissingDF.columns if '_y' in col], axis=1, inplace=True)
MissingDF.head()

,state_po_x,county_name_x,county_fips,candidate_x,party_x,candidatevotes_x,totalvotes_x,UrbanRural,PctRural
67,AK,DISTRICT 1,2001,DONALD J TRUMP,REPUBLICAN,3511.0,7314.0,NaN,NaN
68,AK,DISTRICT 2,2002,DONALD J TRUMP,REPUBLICAN,3674.0,6136.0,NaN,NaN
69,AK,DISTRICT 3,2003,DONALD J TRUMP,REPUBLICAN,6076.0,8385.0,NaN,NaN
70,AK,DISTRICT 4,2004,JOSEPH R BIDEN JR,DEMOCRAT,5345.0,10511.0,NaN,NaN
71,AK,DISTRICT 5,2005,JOSEPH R BIDEN JR,DEMOCRAT,4204.0,8654.0,NaN,NaN


In [26]:
# Look at the extent of the loss
TempDF2 = MissingDF[['state_po_x', 'county_name_x', 'county_fips', 'candidate_x']] \
                        .groupby(['state_po_x', 'county_name_x', 'county_fips']).count()
TempDF2

candidate_x
state_po_x county_name_x county_fips             
AK         DISTRICT 1    2001                   1
           DISTRICT 10   2010                   1
           DISTRICT 11   2011                   1
           DISTRICT 12   2012                   1
           DISTRICT 14   2014                   1
           DISTRICT 15   2015                   1
           DISTRICT 17   2017                   1
           DISTRICT 18   2018                   1
           DISTRICT 19   2019                   1
           DISTRICT 2    2002                   1
           DISTRICT 21   2021                   1
           DISTRICT 22   2022                   1
           DISTRICT 23   2023                   1
           DISTRICT 24   2024                   1
           DISTRICT 25   2025                   1
           DISTRICT 26   2026                   1
           DISTRICT 27   2027                   1
           DISTRICT 28   2028                   1
           DISTRICT 29   2029                   1
           DISTRICT 3    2003                   1
           DISTRICT 30   2030                   1
           DISTRICT 31   2031                   1
           DISTRICT 32   2032                   1
           DISTRICT 33   2033                   1
           DISTRICT 34   2034                   1
           DISTRICT 35   2035                   1
           DISTRICT 36   2036                   1
           DISTRICT 37   2037                   1
           DISTRICT 38   2038                   1
           DISTRICT 39   2039                   1
           DISTRICT 4    2004                   1
           DISTRICT 40   2040                   1
           DISTRICT 5    2005                   1
           DISTRICT 6    2006                   1
           DISTRICT 7    2007                   1
           DISTRICT 8    2008                   1
           DISTRICT 9    2009                   1
           DISTRICT 99   2099                   1
MO         KANSAS CITY   36000                  1
SD         OGLALA LAKOTA 46113                  1

<b>NOTE:</b> Some of the missing data represents small numbers. For the rest:<br>
o- Most are in Alaska. An internet search shows no clear way to confirm the county names or connect the urban/rural designation to the election data.<br>
o- One is for Kansas City, but it isn't clear if this is for the whole city or for the region in Missouri only.

So this data will be dropped. It represents less than 1% of all the data, and the remainder will be enough to investigate the relationships between political ideology and the urban/rural designation.

Reminder that 13 Vermont counties were also dropped above for similar reasons.

## D- Plot the data

In [27]:
# This is the dataframe
PEUrbanRuralDF.head(10)

,state_po,county_name,county_fips,candidate,party,candidatevotes,totalvotes,UrbanRural,PctRural
0,AL,AUTAUGA,1001,DONALD J TRUMP,REPUBLICAN,19838.0,27770.0,urban,42.002162
1,AL,BALDWIN,1003,DONALD J TRUMP,REPUBLICAN,83544.0,109679.0,urban,42.279099
2,AL,BARBOUR,1005,DONALD J TRUMP,REPUBLICAN,5622.0,10518.0,rural,67.789635
3,AL,BIBB,1007,DONALD J TRUMP,REPUBLICAN,7525.0,9595.0,rural,68.352607
4,AL,BLOUNT,1009,DONALD J TRUMP,REPUBLICAN,24711.0,27588.0,rural,89.951502
5,AL,BULLOCK,1011,JOSEPH R BIDEN JR,DEMOCRAT,3446.0,4613.0,rural,51.374382
6,AL,BUTLER,1013,DONALD J TRUMP,REPUBLICAN,5458.0,9488.0,rural,71.232157
7,AL,CALHOUN,1015,DONALD J TRUMP,REPUBLICAN,35101.0,50983.0,urban,33.696826
8,AL,CHAMBERS,1017,DONALD J TRUMP,REPUBLICAN,8753.0,15284.0,urban,49.148034
9,AL,CHEROKEE,1019,DONALD J TRUMP,REPUBLICAN,10583.0,12301.0,rural,85.736273


In [28]:
# Source: https://altair-viz.github.io/gallery/stripplot.html

# Need to disable the error since data is more than 5k rows
alt.data_transformers.disable_max_rows()

domain = ['DEMOCRAT', 'REPUBLICAN']
range_ = ['blue', 'red']

stripplot =  alt.Chart(PEUrbanRuralDF, width=200, height=500).mark_point(filled=True, size=18).encode(
    x=alt.X(
        'jitter:Q',
        title=None,
        axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False),
        scale=alt.Scale(),
    ),
    y=alt.Y('PctRural:Q', title='Percent Rural'),
    color=alt.Color('party:N', scale=alt.Scale(domain=domain, range=range_),
                   title='Party', legend=None),
    size = alt.Size('candidatevotes:Q', scale=alt.Scale(range=[20, 500],zero=False), 
                    title='Candidate Votes', legend=alt.Legend(symbolFillColor='white')),
    column=alt.Column(
        'party:N',
        header=alt.Header(
            labelAngle=0,
            titleOrient='top',
            labelOrient='bottom',
            labelAlign='center',
            labelPadding=30,
            title='Counties by Winning Party',
        ),
    ),
).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)

stripplot

alt.Chart(...)

In [29]:
PEUrbanRuralDF2 = PEUrbanRuralDF.copy()
PEUrbanRuralDF2['fractionvotes'] = PEUrbanRuralDF2['candidatevotes'] / PEUrbanRuralDF2['totalvotes']

alt.Chart(PEUrbanRuralDF2).mark_circle(size=10).encode(
    x='fractionvotes',
    y='PctRural',
    color=alt.Color('party:N', scale=alt.Scale(domain=domain, range=range_),
                   title='Party', legend=None)

)

alt.Chart(...)

In [30]:
PEUrbanRuralDFDem = PEUrbanRuralDF2.copy()
PEUrbanRuralDFDem = PEUrbanRuralDFDem[PEUrbanRuralDFDem['party'] == 'DEMOCRAT']

degree_list = [1, 3, 5]

base = alt.Chart(PEUrbanRuralDFDem).mark_circle(size=10, color="blue").encode(
        alt.X("fractionvotes"), alt.Y("PctRural")
)

polynomial_fit = [
    base.transform_regression(
        "fractionvotes", "PctRural", method="poly", order=order, as_=["fractionvotes", str(order)]
    )
    .mark_line()
    .transform_fold([str(order)], as_=["degree", "PctRural"])
    .encode(alt.Color("degree:N"))
    for order in degree_list
]

alt.layer(base, *polynomial_fit)

alt.LayerChart(...)

In [31]:
PEUrbanRuralDFRep = PEUrbanRuralDF2.copy()
PEUrbanRuralDFRep = PEUrbanRuralDFRep[PEUrbanRuralDFRep['party'] == 'REPUBLICAN']

degree_list = [1, 3, 5]

base = alt.Chart(PEUrbanRuralDFRep).mark_circle(size=10, color="red").encode(
        alt.X("fractionvotes"), alt.Y("PctRural")
)

polynomial_fit = [
    base.transform_regression(
        "fractionvotes", "PctRural", method="poly", order=order, as_=["fractionvotes", str(order)]
    )
    .mark_line()
    .transform_fold([str(order)], as_=["degree", "PctRural"])
    .encode(alt.Color("degree:N"))
    for order in degree_list
]

alt.layer(base, *polynomial_fit)

alt.LayerChart(...)

# II - Urban/Rural Designation and COVID Consequences

In [32]:
# Get full election results data once again.
CountyPresDF = pd.read_csv('../DataForPresidentialElectionsAndCovid/countypres_2000-2020.csv')
print(CountyPresDF.shape)
CountyPresDF.head()

(72603, 12)


,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,AL GORE,DEMOCRAT,4942.0,17208.0,20191203,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993.0,17208.0,20191203,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,RALPH NADER,GREEN,160.0,17208.0,20191203,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,OTHER,OTHER,113.0,17208.0,20191203,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003.0,PRESIDENT,AL GORE,DEMOCRAT,13997.0,56480.0,20191203,TOTAL


In [33]:
# This is the urban/rural designation from above.
print(CountyUrbanRural.shape)
CountyUrbanRural.head()

(3142, 3)


,county_fips,UrbanRural,PctRural
0,1001,urban,42.002162
1,1003,urban,42.279099
2,1005,rural,67.789635
3,1007,rural,68.352607
4,1009,rural,89.951502


In [34]:
# Merge the two to allow filtering by designation.
CountyPresFull = CountyPresDF.merge(CountyUrbanRural, on='county_fips', how='inner')

In [35]:
print(CountyPresFull.shape)
CountyPresFull.sample(5)

(71692, 14)


,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode,UrbanRural,PctRural
30066,2004,MINNESOTA,MN,COTTONWOOD,27033.0,PRESIDENT,JOHN KERRY,DEMOCRAT,2726.0,6369.0,20191203,TOTAL,rural,62.274322
45250,2000,NORTH DAKOTA,ND,CASS,38017.0,PRESIDENT,OTHER,OTHER,1448.0,58995.0,20191203,TOTAL,urban,10.434777
6866,2016,COLORADO,CO,SEDGWICK,8115.0,PRESIDENT,HILLARY CLINTON,DEMOCRAT,267.0,1364.0,20191203,TOTAL,rural,100.000000
10030,2000,GEORGIA,GA,EFFINGHAM,13103.0,PRESIDENT,AL GORE,DEMOCRAT,3232.0,10650.0,20191203,TOTAL,rural,67.052632
17367,2000,INDIANA,IN,NOBLE,18113.0,PRESIDENT,OTHER,OTHER,225.0,14216.0,20191203,TOTAL,rural,68.049478


In [36]:
CountyPresFullUrban = CountyPresFull[CountyPresFull['UrbanRural']=='urban']
CountyPresFullUrban.shape

(28744, 14)

In [37]:
CountyPresFullRural = CountyPresFull[CountyPresFull['UrbanRural']=='rural']
CountyPresFullRural.shape

(42948, 14)

In [38]:
CountyPresFullUrban.to_csv('../DataForPresidentialElectionsAndCovid/CountyPresFullUrban', index=False)

In [39]:
CountyPresFullRural.to_csv('../DataForPresidentialElectionsAndCovid/CountyPresFullRural', index=False)